In [1]:

import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, brier_score_loss, mean_squared_error
from sklearn.calibration import CalibrationDisplay
import numpy as np
import matplotlib.pyplot as plt

In [2]:
YEARS = [2018, 2019, 2020, 2021, 2022, 2023,2024]

# %%
data_all = pd.DataFrame()

def calculate_seconds(row):
    if row['qtr'] != 5:
        return 3600 - row['game_seconds_remaining']
    else:
        return 600 - row['game_seconds_remaining'] + 3600


def get_quarter_value(dataf):
    if 'END QUARTER' in dataf['desc']:
        return dataf['level_0']
    else:
        return None

for i in YEARS:  
    i_data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(i) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data_all = pd.concat([data_all,i_data])

ppr = 1

data = data_all.loc[data_all.season_type=='REG']
#data = data_all.loc[(data_all.play_type.isin(['no_play','pass','run'])) & (data_all.epa.isna()==False)]
#data.loc[data['pass']==1, 'play_type'] = 'pass'
#data.loc[data.rush==1, 'play_type'] = 'run'
data.reset_index(drop=True, inplace=True)
data['turnover'] = data['interception'] + data['fumble_lost']
data = data.dropna(subset=['posteam'])
data['inside_10'] = (data['yardline_100'] < 10).astype(int)
data['20+_play'] = (data['yards_gained'] > 19).astype(int)
data['short_pass'] = (data['air_yards'] < 10).astype(int)
data['medium_pass'] = ((data['air_yards'] > 9)&(data['air_yards']<20)).astype(int)
data['deep_pass'] = (data['air_yards'] > 19).astype(int)
data['end_zone_target'] = (data['yardline_100'] - data['air_yards']) <= 0
data['fantasy_points'] = (
    data['complete_pass'] * ppr +          # 1 point per completion
    data['touchdown'] * 6 +           # 6 points per touchdown
    data['yards_gained'] * 0.1        # 0.1 points per yard gained
)
data['distance_to_EZ_after_target'] = data['yardline_100'] - data['air_yards']


C:\Users\src15\AppData\Local\Temp\ipykernel_6224\2001587006.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['turnover'] = data['interception'] + data['fumble_lost']
C:\Users\src15\AppData\Local\Temp\ipykernel_6224\2001587006.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['turnover'] = data['interception'] + data['fumble_lost']
C:\Users\src15\AppData\Local\Temp\ipykernel_6224\2001587006.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` 

In [3]:
def total_finder(home_or_away,home_total,away_total):
    if home_or_away == 'home':
        total = home_total
    else:
        total = away_total 
    return total

In [5]:
    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]


    # derive implied team total from betting market data
    data['home_implied_total'] = abs(data['total_line'] / 2 + data['spread_line'] / 2)
    data['away_implied_total'] = abs(data['total_line'] / 2 - data['spread_line'] / 2)

    # Use list comprehension with zip for more efficient row-wise operations
    data['implied_posteam_total'] = [
    total_finder(has_ball, home_number, away_number)
        for has_ball, home_number, away_number in zip(data['posteam_type'], data['home_implied_total'], data['away_implied_total'])
    ]

    

C:\Users\src15\AppData\Local\Temp\ipykernel_6224\3013416712.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['home_implied_total'] = abs(data['total_line'] / 2 + data['spread_line'] / 2)
C:\Users\src15\AppData\Local\Temp\ipykernel_6224\3013416712.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['away_implied_total'] = abs(data['total_line'] / 2 - data['spread_line'] / 2)
C:\Users\src15\AppData\Local\Temp\ipykernel_6224\3013416712.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resu

In [6]:
    
    # we only want throws to a receiver, aka plays with air yardage (no running plays, sacks, throwaways etc.)
    throws = data[data['air_yards'].notna()]
    # only data before the current szn
    throws = throws[throws['season']!=2024]
    throws = throws[throws['receiver_player_name'].notna()]
    throws = throws[throws['pass_location'].notna()]

    
    df = throws[['receiver_player_name','receiver_player_id','posteam','pass','cp','game_id','complete_pass','inside_10','air_yards','yardline_100','ydstogo','implied_posteam_total','yards_gained','fantasy_points','pass_touchdown','down','pass_location','week','season','home_implied_total','away_implied_total','posteam_type','qb_hit','end_zone_target', 'distance_to_EZ_after_target']]


In [7]:
df

,receiver_player_name,receiver_player_id,posteam,pass,cp,game_id,complete_pass,inside_10,air_yards,yardline_100,...,down,pass_location,week,season,home_implied_total,away_implied_total,posteam_type,qb_hit,end_zone_target,distance_to_EZ_after_target
2,J.Jones,00-0027944,ATL,1,0.713305,2018_01_ATL_PHI,1.0,0,8.0,80.0,...,1.0,right,1,2018,22.75,21.75,away,0.0,False,72.0
5,C.Ridley,00-0034837,ATL,1,0.766752,2018_01_ATL_PHI,0.0,0,4.0,39.0,...,1.0,right,1,2018,22.75,21.75,away,0.0,False,35.0
6,D.Freeman,00-0031285,ATL,1,0.875661,2018_01_ATL_PHI,0.0,0,-3.0,39.0,...,2.0,left,1,2018,22.75,21.75,away,0.0,False,42.0
7,J.Jones,00-0027944,ATL,1,0.349099,2018_01_ATL_PHI,1.0,0,24.0,39.0,...,3.0,left,1,2018,22.75,21.75,away,0.0,False,15.0
10,D.Freeman,00-0031285,ATL,1,0.610173,2018_01_ATL_PHI,0.0,1,1.0,1.0,...,3.0,right,1,2018,22.75,21.75,away,0.0,True,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263170,M.Evans,00-0031408,TB,1,0.743172,2023_18_TB_CAR,1.0,0,5.0,21.0,...,1.0,right,18,2023,15.75,20.75,away,0.0,False,16.0
263171,T.Palmer,00-0039052,TB,1,0.356356,2023_18_TB_CAR,0.0,0,13.0,15.0,...,2.0,right,18,2023,15.75,20.75,away,1.0,False,2.0
263177,T.Marshall,00-0036955,CAR,1,0.752642,2023_18_TB_CAR,1.0,0,5.0,59.0,...,2.0,left,18,2023,15.75,20.75,home,0.0,False,54.0
263180,D.Chark,00-0034777,CAR,1,0.777339,2023_18_TB_CAR,1.0,0,4.0,52.0,...,1.0,right,18,2023,15.75,20.75,home,0.0,False,48.0


In [10]:


# Load and prepare data (same as before)
predictors = [
    'air_yards', 'yardline_100', 'ydstogo',
    'down', 'pass_location', 'qb_hit', 'end_zone_target', 'distance_to_EZ_after_target'
]
X = df[predictors]
y = df['pass_touchdown']

# Convert categorical variables
X = pd.get_dummies(X, columns=['pass_location'], drop_first=True)
X.fillna(X.mean(), inplace=True)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create XGBoost regression model with probability-focused objective
model = xgb.XGBClassifier(
    objective='binary:logistic',  # Binary classification
    eval_metric=['logloss', 'auc'],
    use_label_encoder=False
)


# Hyperparameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'min_child_weight': [1, 3]
}

# Grid search with both MSE and ROC-AUC scoring
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=['neg_mean_squared_error', 'roc_auc'],
    refit='neg_mean_squared_error',  # Choose MSE as primary metric
    cv=5
)
grid_search.fit(X_train, y_train)

# Best model
touchdown_model = grid_search.best_estimator_

# Make predictions
y_pred = touchdown_model.predict(X_test)

# Evaluate both regression and classification metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
roc_auc = roc_auc_score(y_test, y_pred)

print("Model Evaluation:")
print(f"Best Parameters: {grid_search.best_params_}")
print(f"RMSE: {rmse:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")

# Check calibration across probability ranges
prob_ranges = np.linspace(0, 1, 11)
print("\nProbability Calibration Check:")
for i in range(len(prob_ranges)-1):
    mask = (y_pred >= prob_ranges[i]) & (y_pred < prob_ranges[i+1])
    if mask.any():
        avg_pred_prob = y_pred[mask].mean()
        actual_prob = y_test[mask].mean()
        n_samples = mask.sum()
        print(f"\nProbability range {prob_ranges[i]:.1f}-{prob_ranges[i+1]:.1f} (n={n_samples}):")
        print(f"Average predicted: {avg_pred_prob:.3f}")
        print(f"Actual observed: {actual_prob:.3f}")

# Feature importance
importance = touchdown_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importance:")
print(feature_importance_df)

# Example prediction function
def predict_touchdown_probability(model, play_data):
    """
    Make touchdown probability predictions for new plays.
    """
    # Ensure play_data has same preprocessing as training data
    play_data = pd.get_dummies(play_data, columns=['pass_location'], drop_first=True)
    
    # Add any missing columns from training data
    for col in model.feature_names_in_:
        if col not in play_data.columns:
            play_data[col] = 0
            
    # Reorder columns to match training data
    play_data = play_data[model.feature_names_in_]
    
    # Get predictions and ensure they're in [0,1]
    probabilities = model.predict(play_data)
    probabilities = np.clip(probabilities, 0, 1)
    
    return probabilities


# Print top factors influencing this prediction
def explain_prediction(model, play_data, feature_importance_df):
    """Explain what factors most influenced a specific prediction"""
    # Get the top 5 most important features
    top_features = feature_importance_df.head()
    
    print("\nTop factors influencing this prediction:")
    for _, row in top_features.iterrows():
        feature = row['Feature']
        if feature in play_data.columns:
            value = play_data[feature].iloc[0]
            print(f"{feature}: {value} (importance: {row['Importance']:.3f})")



c:\Users\src15\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:01:42] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\src15\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:01:42] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\src15\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:01:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\src15\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:01:43] WARNING: C:\actio

Model Evaluation:
Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1.0}
RMSE: 0.2166
ROC-AUC Score: 0.5551

Probability Calibration Check:

Probability range 0.0-0.1 (n=20659):
Average predicted: 0.000
Actual observed: 0.042

Feature Importance:
                       Feature  Importance
6  distance_to_EZ_after_target    0.556575
5              end_zone_target    0.257299
1                 yardline_100    0.091014
0                    air_yards    0.040728
2                      ydstogo    0.016648
4                       qb_hit    0.015836
7         pass_location_middle    0.008420
8          pass_location_right    0.006956
3                         down    0.006524


In [11]:
import pickle


with open('touchdown_model.pkl', 'wb') as file:
    pickle.dump(touchdown_model, file)